## Start

In [1]:
datadir = '../data';
wkdir = '../results';

mk_cd_dir(wkdir, false);
%imatlab_export_fig('print-png')

% loocv inputs

prepare_loocv = true

demo_loocv = true

demo_loocv_number_or_list = 'number' % 'number' or 'list'

demo_loocv_number_cmpds = 2

demo_loocv_list_cmpds = {'BRD-K04804440','BRD-K01507359','BRD-K87202646','BRD-K59853741', 'BRD-K27302037'} % Ciprofloxacin, Rifampin, Isoniazid, Q203, Thioacetazone

results_subdir_prefix = 'loocv_pcls/leave_out_cmpd_'

loocv_save_out = false % save tabular file for each treatment's PCL similarity score

unique_kabx_cmpds_tbl_path = '../results/kabx_pert_ids_tbl_for_loocv.txt'

% general inputs

rr_path = fullfile(datadir, 'replicate_correlation.txt')

fgr_path = fullfile(datadir, 'fraction_gr.txt')

% previous Spectral Clustering inputs

thrsh_rank = 20 % threshold for average pairwise rank of correlation across KABX to connect treatments as mutual nearest-neighbors

dynamic_thrsh_per_moa = false % if true then threshold is round(log(size of MOA) * thrsh_rank), otherwise identical threshold for every MOA

k_type = 'k_med_gap_den' % eigengap heuristic to take for estimating number of K clusters: k_num_zero, k_num_zero_plus_one, k_med_gap_den, k_gap_den (see create_laplacian_matrix.m for additional information)

if dynamic_thrsh_per_moa
    outdir_name = sprintf('pcls_spectral_clustering_thrsh_rank_le%dxlogsize_%s', thrsh_rank, k_type) % log(MOA size), i.e. the number of treatments/dsCGI profiles in the MOA
else
    outdir_name = sprintf('pcls_spectral_clustering_thrsh_rank_le%d_%s', thrsh_rank, k_type)
end

outdir = fullfile(wkdir, outdir_name)

mk_cd_dir(outdir, false);


prepare_loocv =

  logical

   1


demo_loocv =

  logical

   1


demo_loocv_number_or_list =

    'number'


demo_loocv_number_cmpds =

     2


demo_loocv_list_cmpds =

  1x5 cell array

  Columns 1 through 3

    {'BRD-K04804440'}    {'BRD-K01507359'}    {'BRD-K87202646'}

  Columns 4 through 5

    {'BRD-K59853741'}    {'BRD-K27302037'}


results_subdir_prefix =

    'loocv_pcls/leave_out_cmpd_'


loocv_save_out =

  logical

   0


unique_kabx_cmpds_tbl_path =

    '../results/kabx_pert_ids_tbl_for_loocv.txt'


rr_path =

    '../data/replicate_correlation.txt'


fgr_path =

    '../data/fraction_gr.txt'


thrsh_rank =

    20


dynamic_thrsh_per_moa =

  logical

   0


k_type =

    'k_med_gap_den'


outdir_name =

    'pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den'


outdir =

    '../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den'



In [2]:
exist(rr_path) > 0
exist(fgr_path) > 0


ans =

  logical

   1


ans =

  logical

   1



## Add replicate reproducibility 

In [3]:
rr = rtable(rr_path);

Reading ../data/replicate_correlation.txt



In [4]:
size(rr)
headt(rr)


ans =

       10819           5


ans =

  5x3 table

    idx           field                                     value                         
    ___    ____________________    _______________________________________________________

     1     {'cid'             }    {'screen_bi1_tbda2:BRD-K04030334-001-01-3:50.000000uM'}
     2     {'count_replicates'}    {[                                                  2]}
     3     {'max_rcorr'       }    {[                                             0.3916]}
     4     {'rcorr'           }    {[                                             0.3916]}
     5     {'rcorr_rank'      }    {[                                             0.0076]}



## Add fraction of GR

In [5]:
fgr = rtable(fgr_path);

Reading ../data/fraction_gr.txt



In [6]:
size(fgr)
headt(fgr)


ans =

       10819           4


ans =

  4x3 table

    idx             field                                 value                   
    ___    ________________________    ___________________________________________

     1     {'cid'                 }    {'kabx2:BRD-A03433762-001-01-3:0.097500uM'}
     2     {'x_median_total_count'}    {[                                 174250]}
     3     {'sum_gr_le0_30'       }    {[                                      0]}
     4     {'frac_gr_le0_30'      }    {[                                      0]}



In [7]:
rr.cid(~ismember(rr.cid, fgr.cid))


ans =

  0x1 empty cell array



In [8]:
fgr.cid(~ismember(fgr.cid, rr.cid))


ans =

  0x1 empty cell array



## Full model section

In [9]:
ls(outdir)

cluster_median_corr_n10819x1947.gctx  pcl_similarity_score_n10819x1140.gctx
ds_corr_n7229x10819.gctx	      pcls.gmt
pcl_and_moa_agree_n10819x1140.gctx



### Add replicate reproducibility and fraction of GR to existing GCTs

In [10]:
% Create gcts
fields = {'cluster_median_corr','pcl_similarity_score','pcl_and_moa_agree'};

for ii = 1:numel(fields)
    disp(fields{ii})
    g = glob(fullfile(outdir,strcat(fields{ii},'_n*.gctx')))
    s = parse_gctx(g{1});
    s.chd
    s.cid(~ismember(s.cid, rr.cid))
    s.cid(~ismember(s.cid, fgr.cid))
    s = annotate_ds(s,table2struct(rr(:,{'cid','rcorr','rcorr_rank'})),'dim','column','keyfield','cid');
    s = annotate_ds(s,table2struct(fgr(:,{'cid','x_median_total_count','frac_gr_le0_30'})),'dim','column','keyfield','cid');
    s.chd
    
    mkgctx(fullfile(outdir,strcat(fields{ii})), s)
end

cluster_median_corr

g =

  1x1 cell array

    {'../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/cluster_median_corr_n10819x1947.gctx'}

Reading ../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/cluster_median_corr_n10819x1947.gctx [1947x10819]
Done [87.83 s].

ans =

  24x1 cell array

    {'broad_id'               }
    {'canonical_smiles'       }
    {'frac_gr_le0_30'         }
    {'pert_class'             }
    {'pert_dose'              }
    {'pert_id'                }
    {'pert_id_pubchem_cid'    }
    {'pert_idose'             }
    {'pert_idose_tickl'       }
    {'pert_iname'             }
    {'pert_reference'         }
    {'pert_type'              }
    {'proj_broad_id'          }
    {'project_id'             }
    {'rcorr'                  }
    {'rcorr_rank'             }
    {'target_description'     }
    {'target_description_long'}
    {'target_pathway'         }
    {'target_process'         }
    {'trt_id'                 }
    {

In [11]:
s.chd
s.rhd


ans =

  24x1 cell array

    {'x_median_total_count'   }
    {'frac_gr_le0_30'         }
    {'rcorr'                  }
    {'rcorr_rank'             }
    {'broad_id'               }
    {'canonical_smiles'       }
    {'pert_class'             }
    {'pert_dose'              }
    {'pert_id'                }
    {'pert_id_pubchem_cid'    }
    {'pert_idose'             }
    {'pert_idose_tickl'       }
    {'pert_iname'             }
    {'pert_reference'         }
    {'pert_type'              }
    {'proj_broad_id'          }
    {'project_id'             }
    {'target_description'     }
    {'target_description_long'}
    {'target_pathway'         }
    {'target_process'         }
    {'trt_id'                 }
    {'x_csv_broad_id'         }
    {'x_subproject_id'        }


ans =

  7x1 cell array

    {'pcl_broad_id'                 }
    {'pcl_desc'                     }
    {'pcl_max_kabx_similarity_score'}
    {'pcl_num_pert_ids_unique'      }
    {'pcl_pert_id'        

## LOOCV section

In [9]:
if prepare_loocv

    unique_kabx_cmpds_tbl = rtable(unique_kabx_cmpds_tbl_path);

    size(unique_kabx_cmpds_tbl)
    headt(unique_kabx_cmpds_tbl)
    
    unique_kabx_cmpds_list = unique(unique_kabx_cmpds_tbl.kabx_cmpd);

    length(unique_kabx_cmpds_list)
    
    number_of_cmpds_loocv = length(unique_kabx_cmpds_list)
    
    if demo_loocv
       if strcmp(demo_loocv_number_or_list, 'number')
           number_of_cmpds_loocv = max(1, demo_loocv_number_cmpds)
           
           index_cmpds_loocv = 1:number_of_cmpds_loocv
           
       elseif strcmp(demo_loocv_number_or_list, 'list')
           number_of_cmpds_loocv = length(demo_loocv_list_cmpds)
           
           index_cmpds_loocv = find(ismember(unique_kabx_cmpds_list, demo_loocv_list_cmpds))'
       else
           error('Invalid input for demo_loocv_number_or_list: number or list')
       end
    end
    
    for i = index_cmpds_loocv
    
        % If the current iteration number is a multiple of 50
        if mod(i, 50) == 0
            % Print a status message
            fprintf('Currently at iteration %d\n', i);
        end

        leave_out_cmpd = unique_kabx_cmpds_list(i);
        
        loo_wkdir = fullfile(wkdir, strcat(results_subdir_prefix, strjoin(unique_kabx_cmpds_list(i))));

        mk_cd_dir(loo_wkdir, false);
        
        loo_outdir = fullfile(loo_wkdir, outdir_name)
        
        mk_cd_dir(loo_outdir, false);
        
        % step specific commands
    
        ls(loo_outdir)
        
        % Add replicate reproducibility and fraction of GR to existing GCTs
        
        % Create gcts
        fields = {'cluster_median_corr','pcl_similarity_score','pcl_and_moa_agree'};

        for ii = 1:numel(fields)
            disp(fields{ii})
            g = glob(fullfile(loo_outdir,strcat(fields{ii},'_n*.gctx')))
            s = parse_gctx(g{1});
            s.chd
            s.cid(~ismember(s.cid, rr.cid))
            s.cid(~ismember(s.cid, fgr.cid))
            s = annotate_ds(s,table2struct(rr(:,{'cid','rcorr','rcorr_rank'})),'dim','column','keyfield','cid');
            s = annotate_ds(s,table2struct(fgr(:,{'cid','x_median_total_count','frac_gr_le0_30'})),'dim','column','keyfield','cid');
            s.chd

            mkgctx(fullfile(loo_outdir,strcat(fields{ii})), s)
        end
        
        s.chd
        s.rhd

    end
    
end

Reading ../results/kabx_pert_ids_tbl_for_loocv.txt


ans =

   437     2


ans =

  2x3 table

    idx          field                value      
    ___    _________________    _________________

     1     {'kabx_cmpd_idx'}    {[            1]}
     2     {'kabx_cmpd'    }    {'BRD-A02179977'}


ans =

   437


number_of_cmpds_loocv =

   437


number_of_cmpds_loocv =

     2


index_cmpds_loocv =

     1     2


loo_outdir =

    '../results/loocv_pcls/leave_out_cmpd_BRD-A02179977/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den'

cluster_median_corr_n9427x1902.gctx  pcl_similarity_score_n9427x1089.gctx
ds_corr_n7234x9427.gctx		     pcls.gmt
pcl_and_moa_agree_n9427x1089.gctx

cluster_median_corr

g =

  1x1 cell array

    {'../results/loocv_pcls/leave_out_cmpd_BRD-A02179977/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/cluster_median_corr_n9427x1902.gctx'}

Reading ../results/loocv_pcls/leave_out_cmpd_BRD-A02179977/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_de